# Setup

In [11]:
#Libraries
import pandas as pd
import censusgeocode as cg
import numpy as np
import random

In [3]:
#Load in the data
addresses = pd.read_csv("data/Allegheny_County_Address_Points.csv", na_values = [""])
census_tracts = pd.read_csv("data/census_variables.csv", na_values = ['NA', '-', '**'])
income_commute = pd.read_csv("data/Alle Co income and commute census data.csv")

D:\Programs\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20,21,26,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Remove useless columns from census_tracts
census_tracts = census_tracts.drop(columns = ["Unnamed: 0", "X"])

# Join using cesus geocode

address -> GEOID

Use the result to join the datasets

In [5]:
#Define a function for taking the address census_tract geocode

def get_geoid(df, address_col, state_col, zip_col, city_muni_col, subset_size, seed):
    '''
    Accepts a data frame and four strings specifying columns names where
    the full address, state, zip code, and city/municipality are stored.
    Any rows with NAs in these columns are filtered. The address attributes are
    extracted, and passed to censusgeocode to retrieve GEOID, which is added
    as a new column. A subset of rows of specified size is used with the 
    specified random seed.
    '''
    # Filter out any rows with NA in address related column
    f_df = df.dropna(subset = [address_col, state_col, zip_col, city_muni_col])
    
    #Select a random subset of rows to get addresses for
    #Dont need all of them for optimization
    subset_f_df = f_df.sample(n = subset_size, random_state = seed)
    
    #Initialize an empty list to collect the GEOIDs
    geoids = []
    
    #Takes a long to run, use a counter to check progress
    total_rows = len(subset_f_df)
    rows_complete = 0
    
    #iterate on each data frame row as a tuple, get address components
    for row in subset_f_df.itertuples():
        full_add = str(getattr(row, address_col))
        state = str(getattr(row, state_col))
        zip_code = str(int(getattr(row, zip_col)))
        city = str(getattr(row, city_muni_col))
        
        #Call censusgeocode for each row to get geoid
        result = cg.address(full_add, city = city, state = state, zipcode = zip_code)
        
        #If a match isn't found, geoid is np.NaN
        if len(result) == 0:
            geoid = np.NaN
        else:
            geoid = result[0]['geographies']['2010 Census Blocks'][0]['GEOID'][0:11]
        
        #Append to the frame
        geoids.append(geoid)
        rows_complete += 1
        
        #Report progress
        prog = round(rows_complete/total_rows, 4)
        print(prog*100,"%")
        
        
    
    #Append to the frame and return
    subset_f_df['GEOID'] = geoids
    
    #Print the NaN Proportion
    nan_prop = subset_f_df['GEOID'].isna().sum()/total_rows
    print('Percent NaN GEOIDs:',nan_prop*100,'%')
    
    #Return the dataframe
    return subset_f_df
        

In [6]:
#Get the frame with GEOID
addresses = get_geoid(addresses, "FULL_ADDRESS", "STATE", "ZIP_CODE", "MUNICIPALITY", 10, 0)

10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 %
Percent NaN GEOIDs: 0.0 %


In [7]:
# Force both GEOID columns to be int for joining
addresses['GEOID'] = addresses['GEOID'].astype(str)
income_commute['GEOID'] = income_commute['GEOID'].astype(str)

# Join the addresses result with income distribution
address_dist = addresses.merge(income_commute, how = "left", left_on = "GEOID", right_on = "GEOID")

In [8]:
# Filter na GEOIDs
address_dist = address_dist.dropna(subset = ['GEOID'])

In [10]:
address_dist.head()

,X,Y,OBJECTID,FEATURE_KEY,ADDRESS_ID,PARENT_ID,STREET_ID,DUP_STREET_ID,ADDRESS_TYPE,STATUS,...,"$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",Total Workers,Commute to work by car,geometry
0,1.351681e+06,417071.311341,495423,495423,495423,NaN,6748,1,Building,Active,...,37,56,75,43,56,34,35,875,489,"list(list(c(-79.974947, -79.968074, -79.96664,..."
1,1.345866e+06,454766.400294,232896,232896,232896,NaN,19806,1,Situs,Active,...,135,168,223,222,220,191,214,2059,1812,"list(list(c(-80.025168, -80.024473, -80.019253..."
2,1.299959e+06,378246.600219,505714,505714,505714,NaN,10838,1,Situs,Active,...,61,157,228,153,141,194,172,2160,1944,"list(list(c(-80.184459, -80.183189, -80.184229..."
3,1.333524e+06,394243.000268,8176,8176,8176,NaN,15403,1,Situs,Active,...,73,94,132,171,44,14,16,1360,1041,"list(list(c(-80.037567, -80.036087, -80.034274..."
4,1.364189e+06,404032.600108,418390,418390,418390,NaN,13057,1,Situs,Active,...,121,162,369,191,218,158,291,2741,1895,"list(list(c(-79.927694, -79.92676, -79.923485,..."


In [74]:
# Create functions to model income and commute probality based on census distributions
def sim_income_commute(df):
    '''
    Simulates income for a data frame based on census income bins.
    '''
    #initialize empty lists
    incomes = []
    commute_statuses = []
    
    for row in df.itertuples():

        #Get the proper attributes
        households = int(getattr(row, "_39"))
        lt_10k = int(getattr(row, "_40"))
        _10_to_14_999 = int(getattr(row, "_41"))
        _15_to_19_999 = int(getattr(row, "_42"))
        _20_to_24_999 = int(getattr(row, "_43"))
        _25_to_29_999 = int(getattr(row, "_44"))
        _30_to_34_999 = int(getattr(row, "_45"))
        _35_to_39_999 = int(getattr(row, "_46"))
        _40_to_44_999 = int(getattr(row, "_47"))
        _45_to_49_999 = int(getattr(row, "_48"))
        _50_to_59_999 = int(getattr(row, "_49"))
        _60_to_75_999 = int(getattr(row, "_50"))
        _75_to_99_999 = int(getattr(row, "_51"))
        _100_to_124_999 = int(getattr(row, "_52"))
        _125_to_149_999 = int(getattr(row, "_53"))
        _150_to_199_999 = int(getattr(row, "_54"))
        gr_200k = int(getattr(row, "_55"))
        total_workers = int(getattr(row, "_56"))
        com_by_car = int(getattr(row, "_57"))
        
        #Simulate income
        #randomly generate a number between 1 and the total households
        #See what bracket that falls into and uniformly generate the income
        samples = [i for i in range(1, households+1)]
        x = random.sample(samples, 1)[0]
        
        #Sampling pointer
        s = 0
        
        if 1 <= x <= lt_10k:
            income = int(random.uniform(1, 9999))
        
        s += lt_10k
            
        if s < x <= (_10_to_14_999+s):
            income = int(random.uniform(10000, 14999))
            
        s += _10_to_14_999
        
        if s < x <= _15_to_19_999+s:
            income = int(random.uniform(15000, 19999))
            
        s += _15_to_19_999
        
        if s < x <= _20_to_24_999+s:
            income = int(random.uniform(20000, 24999))
            
        s += _20_to_24_999
        
        if s < x <= _25_to_29_999+s:
            income = int(random.uniform(25000, 29999))
            
        s + _25_to_29_999
            
        if s < x <= _30_to_34_999+s:
            income = int(random.uniform(30000, 34999))
            
        s += _30_to_34_999
            
        if s < x <= _35_to_39_999+s:
            income = int(random.uniform(35000, 39999))
            
        s += _35_to_39_999
        
        if s < x <= _40_to_44_999+s:
            income = int(random.uniform(40000, 44999))
            
        s += _40_to_44_999
            
        if s < x <= _45_to_49_999+s:
            income = int(random.uniform(45000, 49999))
            
        s += _45_to_49_999
            
        if s < x <= _50_to_59_999+s:
            income = int(random.uniform(50000, 59999))
            
        s += _50_to_59_999
        
        if s < x <=  _60_to_75_999+s:
            income = int(random.uniform(60000, 74999))
            
        s += _60_to_75_999
            
        if s < x <=  _75_to_99_999+s:
            income = int(random.uniform(75000, 99999))
            
        s += _75_to_99_999
            
        if s < x <=  _100_to_124_999+s:
            income = int(random.uniform(100000, 124999))
            
        s += _100_to_124_999
            
        if s < x <=  _125_to_149_999+s:
            income = int(random.uniform(125000, 149999))
            
        s += _125_to_149_999
            
        if s < x <=  _150_to_199_999+s:
            income = int(random.uniform(150000, 199999))
            
        s += _150_to_199_999
            
        if s < x <=  gr_200k+s:
            income = int(random.uniform(200000, 250000))
        
        incomes.append(income)
        
        #Simulate commute status
        #Assume independence from income, except at the highest and lowest levels
        
        if income <= 15000:
            commute = 0
        
        elif income >= 100000:
            commute = 1
        
        else:
            prob_commute = com_by_car / total_workers
            commute = np.random.choice([1,0], 1, p = [prob_commute, 1-prob_commute])[0]
        
        commute_statuses.append(commute)
        
        #Return the resulting lists as a tuple
    return incomes, commute_statuses

In [82]:
#Run the function on our data frame
incomes_list, commute_statuses = sim_income_commute(address_dist)

#Append to the frame
address_dist['simulated_income'] = incomes_list
address_dist['simulated_commute_car'] = commute_statuses

In [83]:
address_dist['simulated_income']

0    141002
1    180080
2     42382
3     56829
4     87783
5     37437
6     54206
7     16788
8    124735
9     92033
Name: simulated_income, dtype: int64

In [85]:
address_dist.columns

Index(['X', 'Y', 'OBJECTID', 'FEATURE_KEY', 'ADDRESS_ID', 'PARENT_ID',
       'STREET_ID', 'DUP_STREET_ID', 'ADDRESS_TYPE', 'STATUS',
       'ADDR_NUM_PREFIX', 'ADDR_NUM', 'ADDR_NUM_SUFFIX', 'ST_PREMODIFIER',
       'ST_PREFIX', 'ST_PRETYPE', 'ST_NAME', 'ST_TYPE', 'ST_POSTMODIFIER',
       'UNIT_TYPE', 'UNIT', 'FLOOR', 'MUNICIPALITY', 'COUNTY', 'STATE',
       'ZIP_CODE', 'ZIP_CODE4', 'COMMENT', 'EDIT_DATE', 'EDIT_USER', 'SOURCE',
       'EXP_FLAG', 'FULL_ADDRESS', 'ENFORCE_VALIDATION', 'SOURCE_ID',
       'ST_SUFFIX', 'GlobalID', 'GEOID', 'Total Households',
       'Less than $10,000', '$10,000 to $14,999', '$15,000 to $19,999',
       '$20,000 to $24,999', '$25,000 to $29,999', '$30,000 to $34,999',
       '$35,000 to $39,999', '$40,000 to $44,999', '$45,000 to $49,999',
       '$50,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999',
       '$100,000 to $124,999', '$125,000 to $149,999', '$150,000 to $199,999',
       '$200,000 or more', 'Total Workers', 'Commute to work by 

In [86]:
address_dist.head()

,X,Y,OBJECTID,FEATURE_KEY,ADDRESS_ID,PARENT_ID,STREET_ID,DUP_STREET_ID,ADDRESS_TYPE,STATUS,...,"$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",Total Workers,Commute to work by car,geometry,simulated_income,simulated_commute_car
0,1.351681e+06,417071.311341,495423,495423,495423,NaN,6748,1,Building,Active,...,75,43,56,34,35,875,489,"list(list(c(-79.974947, -79.968074, -79.96664,...",141002,1
1,1.345866e+06,454766.400294,232896,232896,232896,NaN,19806,1,Situs,Active,...,223,222,220,191,214,2059,1812,"list(list(c(-80.025168, -80.024473, -80.019253...",180080,1
2,1.299959e+06,378246.600219,505714,505714,505714,NaN,10838,1,Situs,Active,...,228,153,141,194,172,2160,1944,"list(list(c(-80.184459, -80.183189, -80.184229...",42382,1
3,1.333524e+06,394243.000268,8176,8176,8176,NaN,15403,1,Situs,Active,...,132,171,44,14,16,1360,1041,"list(list(c(-80.037567, -80.036087, -80.034274...",56829,1
4,1.364189e+06,404032.600108,418390,418390,418390,NaN,13057,1,Situs,Active,...,369,191,218,158,291,2741,1895,"list(list(c(-79.927694, -79.92676, -79.923485,...",87783,1


In [87]:
#Get the relevant columns and write to CSV
cols = ['GEOID','FULL_ADDRESS', 'MUNICIPALITY', 'COUNTY', 'STATE', 'ZIP_CODE', 'simulated_income', 'simulated_commute_car']
final_df = address_dist[cols]

In [88]:
final_df.head()

,GEOID,FULL_ADDRESS,MUNICIPALITY,COUNTY,STATE,ZIP_CODE,simulated_income,simulated_commute_car
0,42003060500,3108 DOBSON ST,PITTSBURGH,ALLEGHENY COUNTY,PA,15219.0,141002,1
1,42003413400,1997 RED COACH RD,MCCANDLESS TOWNSHIP,ALLEGHENY COUNTY,PA,15101.0,180080,1
2,42003456001,568 HICKORY GRADE RD,SOUTH FAYETTE TOWNSHIP,ALLEGHENY COUNTY,PA,15017.0,42382,1
3,42003472300,1223 MCNEILLY AVE,DORMONT BOROUGH,ALLEGHENY COUNTY,PA,15216.0,56829,1
4,42003141400,6655 LANDVIEW RD,PITTSBURGH,ALLEGHENY COUNTY,PA,15217.0,87783,1
5,42003521402,1230 NORTHWESTERN DR,MONROEVILLE MUNICIPALITY,ALLEGHENY COUNTY,PA,15146.0,37437,1
6,42003460001,92 CASTLEVIEW DR,KENNEDY TOWNSHIP,ALLEGHENY COUNTY,PA,15136.0,54206,1
7,42003464400,353 OLIVIA ST,STOWE TOWNSHIP,ALLEGHENY COUNTY,PA,15136.0,16788,0
8,42003270800,1142 GOE AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15212.0,124735,1
9,42003456001,217 NESBIT RD,COLLIER TOWNSHIP,ALLEGHENY COUNTY,PA,15071.0,92033,0


In [91]:
final_df.to_csv('addresses_simulated_incomes.csv', index = False)